In [ ]:
# Importo las librerias
import pandas as pd
import spacy
from tqdm.notebook import tqdm
import re
# si no instalo esto no puedo seguir despues
#!pip install spacy &> /dev/null
#!python -m spacy download es_core_news_lg &> /dev/null
from collections import Counter
import nltk
import seaborn as sns

In [ ]:
df = pd.read_pickle('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX.pkl')
df.info()

In [ ]:
import pandas as pd
from gensim.models import FastText
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
import numpy as np
from sklearn.preprocessing import MinMaxScaler


print("FastText + CatBoost + sin smote")

# Verificación de la distribución de etiquetas
print("Distribución de etiquetas:")
print(df['label'].value_counts())

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Convertir el texto a listas de palabras para entrenar FastText
corpus = [doc.split() for doc in X]

# Entrenar el modelo FastText
ft_model = FastText(sentences=corpus, vector_size=300, window=5, min_count=1, workers=4)

# Crear un vector promedio para cada documento usando las palabras que aparecen en el modelo FastText
def get_fasttext_embeddings(text, model, vector_size):
    # Vector de ceros para las palabras que no están en el vocabulario
    embeddings = np.zeros((len(text), vector_size))

    for i, doc in enumerate(text):
        words = doc.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]

        if len(word_vectors) > 0:
            embeddings[i] = np.mean(word_vectors, axis=0)

    return embeddings

# Crear las representaciones vectoriales de los textos
X_ft = get_fasttext_embeddings(X, ft_model, vector_size=300)

# Verificar los embeddings generados
print("Primeros 5 embeddings generados:")
print(X_ft[:5])  # Revisa las primeras 5 representaciones vectoriales

# Normalizar los vectores para que no haya valores negativos
scaler = MinMaxScaler(feature_range=(0, 1))
X_ft_normalized = scaler.fit_transform(X_ft)

# Definir el modelo de clasificación CatBoost
catboost_model = CatBoostClassifier(cat_features=[], random_seed=42, verbose=0)

# Realizar un ajuste rápido del modelo para verificar las predicciones
catboost_model.fit(X_ft_normalized, y)
predictions = catboost_model.predict(X_ft_normalized)

# Verificar las predicciones iniciales
print("Primeras 10 predicciones:")
print(predictions[:10])

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score, zero_division=1),
    'recall': make_scorer(recall_score, zero_division=1),
    'precision': make_scorer(precision_score, zero_division=1),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usar StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Realizar validación cruzada
final_scores = cross_validate(catboost_model, X_ft_normalized, y, cv=skf, scoring=scoring)
# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE


print("Word2Vec + Random Forest con SMOTE y Optimización de Hiperparámetros")

# Verificación de la distribución de etiquetas
print("Distribución de etiquetas:")
print(df['label'].value_counts())

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Convertir el texto a listas de palabras para entrenar Word2Vec
corpus = [doc.split() for doc in X]

# Entrenar el modelo Word2Vec
w2v_model = Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=1, workers=4)

# Crear un vector promedio para cada documento usando las palabras que aparecen en el modelo Word2Vec
def get_word2vec_embeddings(text, model, vector_size):
    embeddings = np.zeros((len(text), vector_size))

    for i, doc in enumerate(text):
        words = doc.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]

        if len(word_vectors) > 0:
            embeddings[i] = np.mean(word_vectors, axis=0)

    return embeddings

# Crear las representaciones vectoriales de los textos
X_w2v = get_word2vec_embeddings(X, w2v_model, vector_size=300)

# Normalizar los vectores para que no haya valores negativos
scaler = MinMaxScaler(feature_range=(0, 1))
X_w2v_normalized = scaler.fit_transform(X_w2v)

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_w2v_resampled, y_resampled = smote.fit_resample(X_w2v_normalized, y)

# Definir los hiperparámetros a optimizar para Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Definir el modelo de clasificación Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score, zero_division=1),
    'recall': make_scorer(recall_score, zero_division=1),
    'precision': make_scorer(precision_score, zero_division=1),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usar StratifiedKFold para la validación cruzada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Optimización de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           scoring=scoring, refit='f1', cv=skf, verbose=2, n_jobs=-1)

# Realizar la búsqueda de los mejores hiperparámetros
grid_search.fit(X_w2v_resampled, y_resampled)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

# Evaluar el modelo con los mejores hiperparámetros usando validación cruzada
final_scores = cross_validate(grid_search.best_estimator_, X_w2v_resampled, y_resampled, cv=skf, scoring=scoring)

# Imprimir las métricas con desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from gensim.models import FastText
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate

print("FastText + CatBoost con SMOTE")

# Verificación de la distribución de etiquetas
print("Distribución de etiquetas antes de SMOTE:")
print(df['label'].value_counts())

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Convertir el texto a listas de palabras para entrenar FastText
corpus = [doc.split() for doc in X]

# Entrenar el modelo FastText
ft_model = FastText(sentences=corpus, vector_size=300, window=5, min_count=1, workers=4)

# Crear un vector promedio para cada documento usando las palabras que aparecen en el modelo FastText
def get_fasttext_embeddings(text, model, vector_size):
    # Vector de ceros para las palabras que no están en el vocabulario
    embeddings = np.zeros((len(text), vector_size))

    for i, doc in enumerate(text):
        words = doc.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]

        if len(word_vectors) > 0:
            embeddings[i] = np.mean(word_vectors, axis=0)

    return embeddings

# Crear las representaciones vectoriales de los textos
X_ft = get_fasttext_embeddings(X, ft_model, vector_size=300)

# Verificar los embeddings generados
print("Primeros 5 embeddings generados:")
print(X_ft[:5])  # Revisa las primeras 5 representaciones vectoriales

# Normalizar los vectores para que no haya valores negativos
scaler = MinMaxScaler(feature_range=(0, 1))
X_ft_normalized = scaler.fit_transform(X_ft)

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_ft_normalized, y)

# Verificar la nueva distribución de las etiquetas después de SMOTE
print("Distribución de etiquetas después de SMOTE:")
print(pd.Series(y_resampled).value_counts())

# Definir el modelo de clasificación CatBoost
catboost_model = CatBoostClassifier(cat_features=[], random_seed=42, verbose=0)

# Realizar un ajuste rápido del modelo para verificar las predicciones
catboost_model.fit(X_resampled, y_resampled)
predictions = catboost_model.predict(X_resampled)

# Verificar las predicciones iniciales
print("Primeras 10 predicciones:")
print(predictions[:10])

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score, zero_division=1),
    'recall': make_scorer(recall_score, zero_division=1),
    'precision': make_scorer(precision_score, zero_division=1),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usar StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}
# Realizar validación cruzada
final_scores = cross_validate(catboost_model, X_resampled, y_resampled, cv=skf, scoring=scoring)
# Imprimimos los mejores hiperparámetros encontrados
# Entrenamos el modelo con búsqueda de hiperparámetros
#grid_search.fit(X, y)
#print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")
# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")

In [ ]:
df

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("BOW LOGREG SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Regresión Logística
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar el pipeline de imblearn para manejar SMOTE

print("BOW LOGREG CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización, SMOTE y clasificación usando Regresión Logística
pipeline = ImbPipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE para balancear las clases
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar el pipeline de imblearn para manejar SMOTE


print("BOW SVM CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización, SMOTE y clasificación usando SVM
pipeline = ImbPipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE para balancear las clases
    ('svm', SVC(random_state=42))  # Support Vector Machine (SVM)
])

# Definir los hiperparámetros que queremos ajustar en el modelo SVM
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf', 'poly'],  # Tipo de kernel
    'svm__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("BOW SVM SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando SVM (sin SMOTE)
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('svm', SVC(random_state=42))  # Support Vector Machine (SVM)
])

# Definir los hiperparámetros que queremos ajustar en el modelo SVM
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf', 'poly'],  # Tipo de kernel
    'svm__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("BOW Random Forest SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Random Forest (sin SMOTE)
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('rf', RandomForestClassifier(random_state=42))  # Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo Random Forest
param_grid = {
    'rf__n_estimators': [50, 100, 200],  # Número de árboles
    'rf__max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE  # Importar SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar el pipeline de imblearn

print("BOW Random Forest CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Random Forest con SMOTE
pipeline = ImbPipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('smote', SMOTE(random_state=42)),  # SMOTE para balancear las clases
    ('rf', RandomForestClassifier(random_state=42))  # Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo Random Forest
param_grid = {
    'rf__n_estimators': [50, 100, 200],  # Número de árboles
    'rf__max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE  # Importar SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar el pipeline de imblearn

print("BOW Naive Bayes CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Naive Bayes con SMOTE
pipeline = ImbPipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('smote', SMOTE(random_state=42)),  # SMOTE para balancear las clases
    ('nb', MultinomialNB())  # Naive Bayes
])

# Definir los hiperparámetros que queremos ajustar en el modelo Naive Bayes
param_grid = {
    'nb__alpha': [0.01, 0.1, 1, 10],  # Parámetro de suavizado
    'nb__fit_prior': [True, False]  # Ajuste de priors
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("BOW Naive Bayes SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Naive Bayes
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('nb', MultinomialNB())  # Naive Bayes
])

# Definir los hiperparámetros que queremos ajustar en el modelo Naive Bayes
param_grid = {
    'nb__alpha': [0.01, 0.1, 1, 10],  # Parámetro de suavizado
    'nb__fit_prior': [True, False]  # Ajuste de priors
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("BOW CatBoost SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando CatBoost
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('catboost', CatBoostClassifier(learning_rate=0.1, iterations=200, depth=6, random_state=42, verbose=0))  # CatBoost
])

# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Reducimos el número de splits a 3
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Configuramos GridSearchCV con menos combinaciones para acelerar
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("BOW CatBoost CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización, SMOTE y clasificación usando CatBoost
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # Bag of Words (BoW)
    ('smote', SMOTE(random_state=42)),  # SMOTE para balanceo de clases
    ('catboost', CatBoostClassifier(learning_rate=0.1, iterations=200, depth=6, random_state=42, verbose=0))  # CatBoost
])

# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Reducimos el número de splits a 3
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Configuramos GridSearchCV con menos combinaciones para acelerar
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef

print("TF-IDF LOGREG SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Regresión Logística
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE  # Para usar SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Para manejar el pipeline con SMOTE


print("TF-IDF LOGREG CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización, SMOTE y clasificación usando Regresión Logística
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('smote', SMOTE(random_state=42)),  # SMOTE para manejar el desbalance de clases
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.svm import SVC  # Para SVM
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE  # Para usar SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Para manejar el pipeline con SMOTE


print("TF-IDF SVM CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización, SMOTE y clasificación usando SVM
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('smote', SMOTE(random_state=42)),  # SMOTE para manejar el desbalance de clases
    ('svm', SVC(random_state=42))  # Clasificación con SVM
])

# Definir los hiperparámetros que queremos ajustar en el modelo SVM
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf'],  # Tipos de kernel
    'svm__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.svm import SVC  # Para SVM
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("TF-IDF SVM SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando SVM
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('svm', SVC(random_state=42))  # Clasificación con SVM
])

# Definir los hiperparámetros que queremos ajustar en el modelo SVM
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf'],  # Tipos de kernel
    'svm__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier  # Para Random Forest
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("TF-IDF RANDOM FOREST SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando Random Forest
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('rf', RandomForestClassifier(random_state=42))  # Clasificación con Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo Random Forest
param_grid = {
    'rf__n_estimators': [100, 200, 500],  # Número de árboles en el bosque
    'rf__max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5, 10],  # Mínimo número de muestras requeridas para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras requeridas en cada hoja
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier  # Para Random Forest
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE  # Importar SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imblearn


print("TF-IDF RANDOM FOREST CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline con SMOTE para sobremuestreo, TF-IDF y Random Forest
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('smote', SMOTE(random_state=42)),  # Aplicamos SMOTE
    ('rf', RandomForestClassifier(random_state=42))  # Clasificación con Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo Random Forest
param_grid = {
    'rf__n_estimators': [100, 200, 500],  # Número de árboles en el bosque
    'rf__max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5, 10],  # Mínimo número de muestras requeridas para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras requeridas en cada hoja
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  # Para Naive Bayes
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE  # Importar SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imblearn


print("TF-IDF NAIVE BAYES CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline con SMOTE para sobremuestreo, TF-IDF y Naive Bayes
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('smote', SMOTE(random_state=42)),  # Aplicamos SMOTE
    ('nb', MultinomialNB())  # Clasificación con Naive Bayes
])

# Definir los hiperparámetros que queremos ajustar en el modelo Naive Bayes
param_grid = {
    'nb__alpha': [0.1, 0.5, 1.0],  # Parámetro de suavización de Naive Bayes
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Cambiado a TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  # Para Naive Bayes
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.pipeline import Pipeline  # Pipeline de sklearn


print("TF-IDF NAIVE BAYES SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline con TF-IDF y Naive Bayes (sin SMOTE)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Cambiado a TF-IDF
    ('nb', MultinomialNB())  # Clasificación con Naive Bayes
])

# Definir los hiperparámetros que queremos ajustar en el modelo Naive Bayes
param_grid = {
    'nb__alpha': [0.1, 0.5, 1.0],  # Parámetro de suavización de Naive Bayes
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases en la validación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin SMOTE")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("TF-IDF CatBoost CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización, SMOTE y clasificación usando CatBoost
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Usamos TF-IDF en lugar de CountVectorizer
    ('smote', SMOTE(random_state=42)),  # SMOTE para balanceo de clases
    ('catboost', CatBoostClassifier(learning_rate=0.1, iterations=200, depth=6, random_state=42, verbose=0))  # CatBoost
])

# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Reducimos el número de splits a 3
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Configuramos GridSearchCV con menos combinaciones para acelerar
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef


print("TF-IDF CatBoost SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Construimos el pipeline para vectorización y clasificación usando CatBoost
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Usamos TF-IDF en lugar de CountVectorizer
    ('catboost', CatBoostClassifier(learning_rate=0.1, iterations=200, depth=6, random_state=42, verbose=0))  # CatBoost
])

# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Reducimos el número de splits a 3
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Configuramos GridSearchCV con menos combinaciones para acelerar
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


TF-IDF CatBoost SIN SMOTE
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Mejores hiperparámetros: {'catboost__depth': 6, 'catboost__iterations': 100, 'catboost__learning_rate': 0.05} sin smote


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Matthews Correlation Coefficient (MCC): Media = 0.0000, Desvío estándar = 0.0000
F1-score: Media = 0.0000, Desvío estándar = 0.0000
Recall: Media = 0.0000, Desvío estándar = 0.0000
Precision: Media = 0.0000, Desvío estándar = 0.0000


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


print("Word2Vec LOGREG SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras
                return np.mean([model.wv[word] for word in tokens], axis=0)

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline para vectorización y clasificación usando Regresión Logística
pipeline = Pipeline([
    ('word2vec', Word2VecTransformer(vector_size=100)),  # Word2Vec
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np


print("Word2Vec LOGREG CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras
                return np.mean([model.wv[word] for word in tokens], axis=0)

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline con SMOTE y Regresión Logística
pipeline = ImbPipeline([
    ('word2vec', Word2VecTransformer(vector_size=100)),  # Word2Vec
    ('smote', SMOTE(random_state=42)),  # SMOTE para balancear las clases
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from imblearn.over_sampling import SMOTE


print("Word2Vec Naive Bayes CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras y toma el valor absoluto para evitar valores negativos
                return np.abs(np.mean([model.wv[word] for word in tokens], axis=0))

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Creamos una lista de validación cruzada para usarla en el grid search
def grid_search_pipeline(X, y):
    # Aplicamos Word2Vec con el pipeline
    word2vec_transformer = Word2VecTransformer(vector_size=100)
    X_transformed = word2vec_transformer.fit_transform(X)

    # Aplicamos SMOTE sobre los datos transformados
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_transformed, y)

    # Configuramos Naive Bayes
    nb = MultinomialNB()

    # Realizamos la validación cruzada usando el clasificador Naive Bayes
    scoring = {
        'f1': make_scorer(f1_score),
        'recall': make_scorer(recall_score),
        'precision': make_scorer(precision_score),
        'mcc': make_scorer(matthews_corrcoef)
    }

    # Realizamos validación cruzada
    final_scores = cross_validate(nb, X_resampled, y_resampled, cv=skf, scoring=scoring)

    # Imprimir las métricas con su media y desvío estándar
    print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
    print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
    print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
    print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")

# Llamar a la función para hacer el GridSearch y la evaluación
grid_search_pipeline(X, y)


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from imblearn.over_sampling import SMOTE


print("Word2Vec Naive Bayes CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras y toma el valor absoluto para evitar valores negativos
                return np.abs(np.mean([model.wv[word] for word in tokens], axis=0))

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Creamos una lista de validación cruzada para usarla en el grid search
def grid_search_pipeline(X, y):
    # Aplicamos Word2Vec con el pipeline
    word2vec_transformer = Word2VecTransformer(vector_size=100)
    X_transformed = word2vec_transformer.fit_transform(X)

    # Aplicamos SMOTE sobre los datos transformados
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_transformed, y)

    # Configuramos Naive Bayes
    nb = MultinomialNB()

    # Realizamos la validación cruzada usando el clasificador Naive Bayes
    scoring = {
        'f1': make_scorer(f1_score),
        'recall': make_scorer(recall_score),
        'precision': make_scorer(precision_score),
        'mcc': make_scorer(matthews_corrcoef)
    }

    # Realizamos validación cruzada
    final_scores = cross_validate(nb, X_resampled, y_resampled, cv=skf, scoring=scoring)

    # Imprimir las métricas con su media y desvío estándar
    print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
    print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
    print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
    print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")

# Llamar a la función para hacer el GridSearch y la evaluación
grid_search_pipeline(X, y)


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np


print("Word2Vec Random Forest CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras
                return np.mean([model.wv[word] for word in tokens], axis=0)

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline con SMOTE y Random Forest
pipeline = ImbPipeline([
    ('word2vec', Word2VecTransformer(vector_size=100)),  # Word2Vec
    ('smote', SMOTE(random_state=42)),  # SMOTE para balancear las clases
    ('rf', RandomForestClassifier(random_state=42))  # Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Random Forest
param_grid = {
    'rf__n_estimators': [100, 200, 300],  # Número de árboles en el bosque
    'rf__max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5, 10],  # Número mínimo de muestras necesarias para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras necesarias en un nodo hoja
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


print("Word2Vec Naive Bayes SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras y toma el valor absoluto para evitar valores negativos
                return np.abs(np.mean([model.wv[word] for word in tokens], axis=0))

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Creamos una lista de validación cruzada para usarla en el grid search
def grid_search_pipeline(X, y):
    # Aplicamos Word2Vec con el pipeline
    word2vec_transformer = Word2VecTransformer(vector_size=100)
    X_transformed = word2vec_transformer.fit_transform(X)

    # Configuramos Naive Bayes
    nb = MultinomialNB()

    # Realizamos la validación cruzada usando el clasificador Naive Bayes
    scoring = {
        'f1': make_scorer(f1_score),
        'recall': make_scorer(recall_score),
        'precision': make_scorer(precision_score),
        'mcc': make_scorer(matthews_corrcoef)
    }

    # Realizamos validación cruzada
    final_scores = cross_validate(nb, X_transformed, y, cv=skf, scoring=scoring)

    # Imprimir las métricas con su media y desvío estándar
    print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
    print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
    print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
    print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")

# Llamar a la función para hacer el GridSearch y la evaluación
grid_search_pipeline(X, y)


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


print("Word2Vec Random Forest SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras
                return np.mean([model.wv[word] for word in tokens], axis=0)

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline sin SMOTE y con Random Forest
pipeline = Pipeline([
    ('word2vec', Word2VecTransformer(vector_size=100)),  # Word2Vec
    ('rf', RandomForestClassifier(random_state=42))  # Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Random Forest
param_grid = {
    'rf__n_estimators': [100, 200, 300],  # Número de árboles en el bosque
    'rf__max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5, 10],  # Número mínimo de muestras necesarias para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras necesarias en un nodo hoja
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


print("Word2Vec CatBoost CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras
                return np.mean([model.wv[word] for word in tokens], axis=0)

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline para Word2Vec, SMOTE y clasificación usando CatBoost
pipeline = Pipeline([
    ('word2vec', Word2VecTransformer(vector_size=100)),  # Word2Vec
    ('smote', SMOTE(random_state=42)),  # SMOTE para balanceo de clases
    ('catboost', CatBoostClassifier(learning_rate=0.1, iterations=200, depth=6, random_state=42, verbose=0))  # CatBoost
])

# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Reducimos el número de splits a 3
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Configuramos GridSearchCV con menos combinaciones para acelerar
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from imblearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


print("Word2Vec CatBoost SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando Word2Vec
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo Word2Vec
        sentences = [sentence.split() for sentence in X]
        self.model = Word2Vec(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_word2vec(tokens, model, vector_size):
            # Elimina palabras desconocidas del modelo
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                # Promedia los vectores de las palabras
                return np.mean([model.wv[word] for word in tokens], axis=0)

        # Transformamos el conjunto de datos
        return np.array([get_average_word2vec(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline para Word2Vec y clasificación usando CatBoost (sin SMOTE)
pipeline = Pipeline([
    ('word2vec', Word2VecTransformer(vector_size=100)),  # Word2Vec
    ('catboost', CatBoostClassifier(learning_rate=0.1, iterations=200, depth=6, random_state=42, verbose=0))  # CatBoost
])

# Reducimos el rango de hiperparámetros
param_grid = {
    'catboost__learning_rate': [0.05, 0.1],  # Reducimos el rango de tasas de aprendizaje
    'catboost__iterations': [100, 200],  # Menos iteraciones para acelerar
    'catboost__depth': [6]  # Usamos un solo valor de profundidad para acortar
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Reducimos el número de splits a 3
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Configuramos GridSearchCV con menos combinaciones para acelerar
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText LOGREG SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Construimos el pipeline para vectorización y clasificación usando Regresión Logística
pipeline = Pipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X, y)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X, y, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText LOGREG CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos antes de SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización, sobremuestreo con SMOTE y clasificación usando Regresión Logística
pipeline = ImbPipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE solo en el entrenamiento
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Regresión Logística
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Regresión Logística
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'logreg__solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'logreg__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText SVM CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos antes de SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización, sobremuestreo con SMOTE y clasificación usando SVM
pipeline = ImbPipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE solo en el entrenamiento
    ('svm', SVC(random_state=42))  # Clasificación usando SVM
])

# Definir los hiperparámetros que queremos ajustar en el modelo de SVM
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf'],  # Tipos de kernel
    'svm__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText RANDOM FOREST CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos antes de aplicar SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización, sobremuestreo con SMOTE y clasificación usando Random Forest
pipeline = ImbPipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE solo en el entrenamiento
    ('rf', RandomForestClassifier(random_state=42))  # Clasificación usando Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Random Forest
param_grid = {
    'rf__n_estimators': [100, 200, 500],  # Número de árboles en el bosque
    'rf__max_depth': [10, 20, None],  # Máxima profundidad del árbol
    'rf__min_samples_split': [2, 5, 10],  # Mínimo número de muestras requeridas para dividir un nodo
    'rf__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler


print("FastText NAIVE BAYES CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos antes de aplicar SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización, sobremuestreo con SMOTE y clasificación usando Naive Bayes
pipeline = ImbPipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('scaler', MinMaxScaler()),  # Escalar los datos a [0, 1] para Naive Bayes
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE solo en el entrenamiento
    ('nb', MultinomialNB())  # Clasificación usando Naive Bayes
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Naive Bayes
param_grid = {
    'nb__alpha': [0.1, 0.5, 1.0],  # Suavizado de Laplace
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from imblearn.pipeline import Pipeline as ImbPipeline  # Usar Pipeline de imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler


print("FastText CATBOOST CON SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos antes de aplicar SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización, sobremuestreo con SMOTE y clasificación usando CatBoost
pipeline = ImbPipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('smote', SMOTE(random_state=42)),  # Aplicar SMOTE solo en el entrenamiento
    ('catboost', CatBoostClassifier(random_state=42, verbose=0))  # Clasificación usando CatBoost
])

# Definir los hiperparámetros que queremos ajustar en el modelo CatBoost
param_grid = {
    'catboost__iterations': [100, 200],
    'catboost__learning_rate': [0.01, 0.1],
    'catboost__depth': [4, 6, 8],  # Profundidad de los árboles
    'catboost__l2_leaf_reg': [1, 3, 5]  # Regularización L2
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} con smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin

print("FastText SVM SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización y clasificación usando SVM
pipeline = Pipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('svm', SVC(random_state=42))  # Clasificación usando SVM
])

# Definir los hiperparámetros que queremos ajustar en el modelo de SVM
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf'],  # Tipos de kernel
    'svm__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_} sin smote")

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText Naive Bayes SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización y clasificación usando Naive Bayes
pipeline = Pipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('naive_bayes', GaussianNB())  # Clasificación usando Naive Bayes
])

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada (sin hiperparámetros específicos en Naive Bayes)
grid_search = GridSearchCV(pipeline, param_grid={}, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText Random Forest SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización y clasificación usando Random Forest
pipeline = Pipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('random_forest', RandomForestClassifier(random_state=42))  # Clasificación usando Random Forest
])

# Definir los hiperparámetros que queremos ajustar en el modelo de Random Forest
param_grid = {
    'random_forest__n_estimators': [100, 200],  # Número de árboles en el bosque
    'random_forest__max_depth': [None, 10, 20],  # Profundidad máxima de los árboles
    'random_forest__min_samples_split': [2, 5],  # Número mínimo de muestras necesarias para dividir un nodo
    'random_forest__min_samples_leaf': [1, 2],  # Número mínimo de muestras necesarias en una hoja
    'random_forest__class_weight': [None, 'balanced']  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, matthews_corrcoef
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin


print("FastText CatBoost SIN SMOTE")

X = df['text_clean']  # Texto procesado
y = df['label']  # Etiquetas de sesgo

# Crea un transformador personalizado para convertir texto a vectores usando FastText
class FastTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        # Tokenizamos el texto y entrenamos el modelo FastText
        sentences = [sentence.split() for sentence in X]
        self.model = FastText(sentences, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=4)
        return self

    def transform(self, X):
        # Transforma el texto en vectores promediados de las palabras
        def get_average_fasttext(tokens, model, vector_size):
            tokens = [word for word in tokens if word in model.wv]
            if len(tokens) == 0:
                return np.zeros(vector_size)
            else:
                return np.mean([model.wv[word] for word in tokens], axis=0)

        return np.array([get_average_fasttext(sentence.split(), self.model, self.vector_size) for sentence in X])

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construimos el pipeline para vectorización y clasificación usando CatBoost
pipeline = Pipeline([
    ('fasttext', FastTextTransformer(vector_size=100)),  # FastText embeddings
    ('catboost', CatBoostClassifier(verbose=0, random_state=42))  # Clasificación usando CatBoost
])

# Definir los hiperparámetros que queremos ajustar en el modelo de CatBoost
param_grid = {
    'catboost__iterations': [100, 200],  # Número de iteraciones o árboles
    'catboost__depth': [6, 8, 10],  # Profundidad máxima del árbol
    'catboost__learning_rate': [0.01, 0.1],  # Tasa de aprendizaje
    'catboost__l2_leaf_reg': [1, 3, 5],  # Regularización L2
    'catboost__border_count': [32, 64],  # Número de puntos de división en las características continuas
    'catboost__class_weights': [None, {0: 1, 1: 2}]  # Ajuste de peso de clases
}

# Definir las métricas que queremos evaluar
scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'mcc': make_scorer(matthews_corrcoef)  # Matthew's Correlation Coefficient
}

# Usamos StratifiedKFold para manejar el desbalance de clases
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1', refit=True, verbose=1)

# Entrenamos el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Mejor estimador
best_model = grid_search.best_estimator_

# Realizamos validación cruzada para evaluación final con múltiples métricas
final_scores = cross_validate(best_model, X_test, y_test, cv=skf, scoring=scoring)

# Imprimir las métricas con su media y desvío estándar
print(f"Matthews Correlation Coefficient (MCC): Media = {final_scores['test_mcc'].mean():.4f}, Desvío estándar = {final_scores['test_mcc'].std():.4f}")
print(f"F1-score: Media = {final_scores['test_f1'].mean():.4f}, Desvío estándar = {final_scores['test_f1'].std():.4f}")
print(f"Recall: Media = {final_scores['test_recall'].mean():.4f}, Desvío estándar = {final_scores['test_recall'].std():.4f}")
print(f"Precision: Media = {final_scores['test_precision'].mean():.4f}, Desvío estándar = {final_scores['test_precision'].std():.4f}")
